# Практика

## Задача

Есть переписка ТП с пользователем.
Переписка содержит все данные по решению инцидента
- описание и уточнение проблемы,
- обсуждение вариантов решения,
- итоговое решение

На основе переписки формировать два текста:
1. краткое описание проблемы (возможно, с выделением ключевых слов),
2. шаги решения (инструкция).

Будет использоваться механизм поиска похожих обращений (по описанию проблемы)

## Ожидаемое решение

Функция принимающая на вход `.docx`-файлы, и возвращающая массив решений (и пусть директорию решений в `.txt`-файлах).

## Этапы решения задачи.

### 1. Загрузка `.docx`-документов.

Используя библиотеку `python-docx` для обработки Word-документов, описана функция `load_docx_from_project(path)` для гибкой загрузки всех `.docx`-файлов из заданной директории, от моментального форматирования в класс `Chat` отказался в пользу масштабируемости, чтобы на руках были сырые данные.

### 2. Представление и формат данных.

Данные в переписке представлены как таблицы с колонками:
- Номер обращения
- Отправитель
- Текст сообщения

Решение строится на 3х ключевых классах:

- `Message`: инкапсулирует отправителя и текст.
- `Chat`: объединяет список сообщений и метаданные (название, номера обращений).
- `CompanyChat`: содержит очищенный текст чата, готовый к отправке в ИИ.

**Плюсы:**
- Чёткое разделение областей ответственности.
- Упрощает последующую обработку данных.
- Повышает читаемость и поддержку кода.
Отсюда вытекает небольшой выигрыш в гибкости и мобильности кода.

### 3. Очистка и нормализация текста.

Перед обработкой ИИ, данные проходят классическую фильтрацию:
- Удаление спецсимволов.
- Очистка пустых строк.
- Приведение регистра (опционально).


### 4. Подготовка промптов для ИИ.

Для каждого типа запроса формируется `LLMRequest` на базе `InstructionBlock`, который в свою очередь будет формироваться из `.json`-файла содержащего:
- Роль (например: "Ты — ассистент, анализирующий переписку...")
- Инструкцию
- Контекст (например, список аббревиатур)
- Формат ответа
- `max_tokens` — ограничение длины ответа
Такой подход должен обеспечить возможность к будущему расширению и простоте управления (исправления).

### 5. Отправка запроса и обработка со стороны ИИ.

Создается запрос к заданному ИИ с инструктивным промптом к задаче. Использован `ThreadPoolExecutor`, чтобы одновременно отправлять три запроса:
- Краткое описание
- Ключевые слова
- Решение проблемы
Выбор пал именно на `ThreadPoolExecutor` так как в отличии, например, от `Threading` дает простое управление потоками, и главное - его не блокирует GIL, так как выполняются операции I/O.

**Плюсы:**
- Экономия времени при большом числе файлов.
- Возможность легко отключить параллельность для отладки (флаг `multi_thread`).

### 6. Сохранение результатов и финальная обертка.

После обработки создаётся объект `ProblemWithSolution`, в который входят:
- Название компании
- Краткое описание
- Ключевые слова
- Инструкция

Отдельно от `.ipynb`-решения создам модуль, с функцией, который будет на вход получать директорию и возвращать директорию с `.txt` решениями.

## Заключение.

Так как задача оказалось достаточно простой, я прибегнул к упрощению и оптимизации отдельных фрагментов. Таким образом удалось достичь:
- **Гибкости**: в код достаточно много разделений на отдельный элементы, классы или функции, что позволит адаптировать под новые задачи.
- **Масштабируемости**: реализация сразу предполагает работу как с одним документом, так и с массивом.
- **Поддержки повторного использования**: структуры `InstructionBlock`, `LLMRequest`, `Chat`, `CompanyChat` легко масштабируются и тестируются.
- **Расширяемости**: добавление новых типов инструкций (например, тональность, оценка качества и т.п) - это лишь вопрос дополнительных полей инструкции.

### 1. Начало работы
- Из директории ```input_data```, содержащую в себе .docx файлы, необходимо извлечь данные.
- С извлечением поможет библиотека ```python_docx```.
- Для извлечения опишу функцию, которая вернет сырые данные (возможно пригодится в плане гибкости).

In [54]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
from os import listdir, path
from docx import Document


def load_docx_from_project(path_in_project='input_data/'):
    documents = []

    for filename in listdir(path_in_project):
        if filename.endswith('.docx'):
            full_path = path.join(path_in_project, filename)
            try:
                document = Document(full_path)
                documents.append([filename, document])
            except Exception as e:
                print(f"Не удалось загрузить {filename}: {e}")

    return documents


# Извлечение данных из файлов .docx
docs = load_docx_from_project()


### 2. Обработка входных данных
- Входные данные представлены в формате .docx -> их извлечение дает свой особый формат.
- Необходимо обработать данные путем прохода по строкам и столбцам таблицы. Также нужно очистить данные от мусора для ИИ.
- Для обработки следует создать 3 класса: Message, Chat и Company_Chat:
    - Message хранит в себе отправитель сообщения и его текст.
    - Chat содержит в себе массив Messageй, имя компании, и номера обращений (возможно понадобятся в плане гибкости).
    - CompanyChat - очищенный чат с компанией (название, переписка).
- После обработки хорошим решением кажется создания директории очищенных .txt файлов.

##### Создание классов ```Message```, ```Chat```, ```Company_Chat```.

In [56]:
from dataclasses import dataclass


@dataclass
class Message:
    sender: str
    text: str


@dataclass
class Chat:
    messages: list[Message]
    name: str = None
    numbers: list[str] = None


@dataclass
class CompanyChat:
    company: str
    whole_chat: str


##### Конвертация данных из .docx в Chat.

In [57]:
def convert_from_docx(document: tuple[str, Document]):
    name = document[0][:-5]

    table = document[1].tables[0]
    numbers: list[str] = []
    messages: list[Message] = []
    for row in table.rows[1:]:
        cells = row.cells
        message = Message(cells[1].text, cells[2].text)

        numbers.append(cells[0].text)
        messages.append(message)
    numbers = set(numbers)

    chat = Chat(
        name=name,
        numbers=numbers,
        messages=messages
    )
    '''
    print(f"|{'Компания':<16}|", chat.name,
          f"\n|{'№ обращений':<16}|", chat.numbers,
          f"\n|{'Отправитель':<16}|", chat.messages[0].sender,
          f"\n|{'Текст сообщения':<16}|\n", chat.messages[0].text)
    '''
    return chat


# Изменение типа переменных
chats: list[Chat] = []
for doc in docs:
    chat = convert_from_docx(doc)
    chats.append(chat)


##### Очистка Chatов, сохранение в массив CompanyChat.

In [58]:
from re import sub


REMOVE_CHARS = {
    0x00A0: None,  # NBSP – неразрывный пробел
    0x00AD: None,  # SHY – мягкий перенос
    0x200B: None,  # ZWSP – нулевой пробел
    0xFEFF: None   # BOM – метка порядка байтов
}


def clear_text(
    text: str,
    symbols_to_remove: dict[int, None],
    to_lower: bool = False,
    normalize_spaces: bool = True
) -> str:
    text = text.translate(symbols_to_remove)
    text = sub(r'\n+', '\n', text)
    text = sub(r'[ \t]+\n', '\n', text)

    if normalize_spaces:
        text = sub(r'[ \t]{2,}', ' ', text)

    text = "\n".join(line.strip() for line in text.splitlines())

    if to_lower:
        text = text.lower()

    return text.strip()


def format_chat(chat: Chat) -> CompanyChat:
    dialog = ""

    for message in chat.messages:
        cleared_text = clear_text(message.text, REMOVE_CHARS)
        dialog += (message.sender + ': ' + '\n' +
                   cleared_text + '\n\n')

    name_data = CompanyChat(chat.name, dialog)
    return name_data


texts_to_process: list[CompanyChat] = []
for chat in chats:
    texts_to_process.append(format_chat(chat))


##### Подготовка директории и загрузка в нее готовых данных.

In [59]:
from os import makedirs


def ensure_folder_exists(folder_path: str = "to_process") -> None:
    makedirs(folder_path, exist_ok=True)


def write_text_to_file(
    data: CompanyChat,
    folder_path: str = "to_process"
) -> None:
    file_path = folder_path + '/' + data.company + ".txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(data.whole_chat)


ensure_folder_exists()
for data in texts_to_process:
    write_text_to_file(data)


### 3. Основной фрагмент
Данные очищены, отформатированы и лежат на низком старте. Для их обработки нужно формировать запросы к ИИ. Запросы будут формироваться из инструкций и переписки с ТП. Для всего озвученного целесообразно снова прибегнуть к классам:
- ```InstructionBlock```    - составной блок инструкций для ИИ;
- ```LLMRequest```          - запрос из инструкций и задачи;
- ```AIModelAPI```          - API к модели ИИ;
- ```ProblemWithSolutin```  - данные на выход (компания, краткое описание проблемы, ключевые слова, структурированное решение).

Решение будет разбито на пару шагов:
- До отправки запрос пройдет фазу сборки из инструкции и текста чата. Формирование инструкции лучше всего производить через .json файл, такой подход:
     - Увеличивает мобильность и гибкость кода.
     - Упрощает коррекцию инструкций.
     - Уменьшает и упрощает код.

- По API будут параллельно отправлены 3 запроса к ИИ (DeepSeek V3/ ChatGPT 4 turbo):
    - Формирование краткого описания проблемы.
    - Извлечение ключевых слов.
    - Поиск и создание решения.

- Переменные класса ProblemWithSolution будут сформированы из названия файла, хранящего переписку с ТП, и из ответов на запросы.

##### Создание классов - ```InstructionBlock```, ```LLMRequest```, ```AIModelAPI```, ```ProblemWithSolutin```.

In [60]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
from json import load
from dataclasses import dataclass
from openai import OpenAI


@dataclass
class InstructionBlock:
    role: str
    instruction: str
    context: str
    format: str | None
    max_tokens: int


@dataclass
class LLMRequest:
    instruction_block: InstructionBlock
    task: str

    def to_prompt(self) -> list[dict[str, str]]:
        prompt = (
            f"{self.instruction_block.instruction}\n\n"
            f"Контекст:\n{self.instruction_block.context}\n\n"
            f"Формат вывода:\n{self.instruction_block.format}\n\n"
            f"Ограничение по токенам:\n{self.instruction_block.max_tokens}\n\n"
            f"Входные данные:\n{self.task}"
        )
        return [
            {"role": "system", "content": self.instruction_block.role},
            {"role": "user", "content": prompt}
        ]

    @property
    def max_tokens(self) -> int:
        return self.instruction_block.max_tokens


class AIModelAPI:
    def __init__(self, api: str, url: str, model_name: str):
        self.api = api
        self.url = url
        self.model_name = model_name
        self.client = OpenAI(api_key=self.api, base_url=self.url)

    def get_response(self, request: LLMRequest,
                     max_tokens: int = 50, temperature: float = 0.1):
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=request.to_prompt(),
            stream=False,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content


@dataclass
class ProblemWithSolution:
        description: str
        keywords: list[str]
        solution: str
        name: str = "untitled"
        numbers: list[str] = None


##### Извлечение данных .txt, .json (переписки ТП, конфигурация инструкций). Формирование инструкций.

In [62]:
def load_data(instruction_path: str) -> str:
    with open(instruction_path, "r", encoding="utf-8") as f:
        return f.read()


def load_instruction_file(json_path: str):
    with open(json_path, "r", encoding="utf-8") as f:
        data = load(f)

    abbreviations = "\n".join(data["context"])
    role = data["role"]

    def make_instruction(key: str) -> InstructionBlock:
        item = data["instructions"][key]
        return InstructionBlock(
            role=role,
            instruction=item["instruction"],
            context=abbreviations,
            format=item["response_format"],
            max_tokens=item["max_tokens"]
        )

    return {
        "abbreviations": abbreviations,
        "instruction_for_description": make_instruction("description"),
        "instruction_for_keywords": make_instruction("keywords"),
        "instruction_for_solution": make_instruction("solution"),
    }

##### Основная функция формирующая 3 инструктивных запроса с задачей и возвращающая переменную класса решения.

In [63]:
from concurrent.futures import ThreadPoolExecutor, as_completed

  
def chat_process(
        model: AIModelAPI,
        path_to_instruct_json: str,
        path_to_task_txt: str,
        multi_thread: bool = False
) -> ProblemWithSolution:

    def _call_request(request: LLMRequest) -> str:
        result = model.get_response(request, max_tokens=request.max_tokens)
        return result

    def _check_json_and_txt(path_json: str, path_to_txt: str,) -> None:
        shaped_path_task = path.splitext(path.basename(path_to_txt))
        format_of_file_task = shaped_path_task[1]
        if format_of_file_task != ".txt":
            raise ValueError(f"Ожидаемый формат файла задачи - .txt, получен - {format_of_file_task}")
        
        shaped_path_instruct = path.splitext(path.basename(path_json))
        format_of_file_instruct = shaped_path_instruct[1]
        if format_of_file_instruct != ".json":
            raise ValueError(f"Ожидаемый формат файла инструкций - .json, получен - {format_of_file_task}")
    
    
    _check_json_and_txt(path_json=path_to_instruct_json, path_to_txt=path_to_task_txt)
    
    name = path.splitext(path.basename(path_to_task_txt))[0]
    
    general_task = load_data(path_to_task_txt)

    instructions = load_instruction_file(path_to_instruct_json)
    i_description = instructions["instruction_for_description"]
    i_keywords    = instructions["instruction_for_keywords"]
    i_solution    = instructions["instruction_for_solution"]

    r_description = LLMRequest(i_description, task=general_task)
    r_keywords = LLMRequest(i_keywords,       task=general_task)
    r_solution = LLMRequest(i_solution,       task=general_task)

    tasks = {
        "description": r_description,
        "keywords":    r_keywords,
        "solution":    r_solution
    }

    results: dict[str, str] = {}

    if multi_thread:
        with ThreadPoolExecutor(max_workers=3) as executor:
            future_to_label = {
                executor.submit(_call_request, request): label
                for label, request in tasks.items()
            }
            for future in as_completed(future_to_label):
                label = future_to_label[future]
                try:
                    results[label] = future.result(timeout=30)
                except Exception as e:
                    results[label] = f"Ошибка: {e}"

    else:
        for label, request in tasks.items():
            try:
                results[label] = _call_request(request)
            except Exception as e:
                results[label] = f"Ошибка: {e}"


    for key in tasks: results.setdefault(key, "")

    return ProblemWithSolution(
        name=name,
        description=results["description"],
        keywords=results["keywords"].split(),
        solution=results["solution"]
    )


### 4. Загрузка данных и библиотек и тестовый запуск.

##### Загрузка конфигурационных данных.

In [64]:
from os import getenv
from dotenv import load_dotenv


load_dotenv()

path_to_task_txt = "to_process/ПАКС ООО.txt"
path_to_instruct_json = "prompt_data.json"

model_url = "https://api.deepseek.com"
model_name = "deepseek-chat"
API_AI = getenv("API_DS")

# model_url = "https://api.openai.com/v1"
# model_name = "gpt-4-turbo"
# API_AI = getenv("API_GPT")

model = AIModelAPI(API_AI, model_url, model_name)


##### Запуск и отладочный вывод.

In [65]:
sol = chat_process(
    model=model,
    path_to_instruct_json=path_to_instruct_json,
    path_to_task_txt=path_to_task_txt,
    multi_thread=False
)

separator = "-" * 50

print(f"{sol.name}              \n{separator}\n"
      f"{sol.description}       \n{separator}\n"
      f"{' '.join(sol.keywords)}\n{separator}\n"
      f"{sol.solution}"                         )


KeyboardInterrupt



### 5. Описание единой функции будет создано в отдельном проекте.